# Aufgabe 1

In [243]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numbers
import math
import matplotlib.pyplot as plt
import numpy as np

class Polynom:
    ''' Polynom-Klasse '''

    def __init__(self, dp, alg):

        # Entferne alle Terme mit Koeffizienten 0
        self.dp = {k: v for k, v in dp.items() if v != 0}
        # falls dadurch dp leer wird, müssen wri dp auf das Nullpolynom setzen
        if not self.dp:
            self.dp = {0: 0}

        self.alg = alg
        self.degree = max(self.dp.keys())
        # Die Überprüfung der Eingabe wurde in die __call__ Methode von alg verschoben

    def _pstr(self):
        ''' Hilfsfunktion zum Erstellen des Polynomstring 
            Die geschweiften Klammern um den Exponenten brauchen wir 
            für die Legende im Plot
        '''
        polystr = ''
        for k, pk in reversed(sorted(self.dp.items())):
            if isinstance(pk, complex):  # Klammern um komplexe Zahl
                polystr = polystr + f'+({pk:.3f})*X^{{{k}}}'
            elif round(pk) == pk:
                polystr = polystr + f'+{pk:g}*X^{{{k}}}'
            else:
                polystr = polystr + f'+{pk:.3f}*X^{{{k}}}'
        replacements = {"*X^{0}": "", "X^{1}": "X",
                        "+-1*": "-", "+1*": "+", "+-": "-"}
        for k, v in replacements.items():
            polystr = polystr.replace(k, v)
        polystr = polystr.lstrip('+')
        return polystr

    def _getfield(self, a, b):
        '''
        Hilfsfunktion um den Typ der algebraischen Struktur anzupassen.
        So soll die Addition von einem Polynom in C[X] und einem in Z[X] 
        erlaubt sein und ein Polynom in C[X] liefern.
        '''
        if a.field is numbers.Complex or b.field is numbers.Complex:
            field_str_ = 'C'
        elif a.field is numbers.Real or b.field is numbers.Real:
            field_str_ = 'R'
        else:
            field_str_ = 'Z'
        #if hasattr(a, 'dim'): # fuer Vektorraeume
        #    return field_str_, a.dim
        return field_str_
        
    def __repr__(self):
        polystr = self._pstr()
        return f'{__class__.__name__} in  {self.alg}  {polystr}'

    def __str__(self):
        polystr = self._pstr()
        return f'{__class__.__name__} {polystr}'

    def __add__(self, other):
        ''' 
            Addition von Polynomen, damit
            für Polynome p und q  p+q die Summe berechnet
        '''
        if isinstance(other, numbers.Number):
            other = self.alg(other)
        Kdim = self._getfield(self.alg,other.alg)
        alg_ = self.alg.__class__(*Kdim)
        return self.__class__(self.alg.add(self, other), alg_)

    def __radd__(self, other):
        ''' 
            Addition von Polynomen, damit
            für eine Zahl p und ein Polynom q
            auch p+q funktioniert
        '''
        if isinstance(other, numbers.Number):
            other = self.alg(other)
        Kdim = self._getfield(self.alg,other.alg)
        alg_ = self.alg.__class__(*Kdim)            
        return self.__class__(self.alg.add(other, self), alg_)

    def __sub__(self, other):
        if isinstance(other, numbers.Number):
            other = self.alg(other)
        Kdim = self._getfield(self.alg,other.alg)
        alg_ = self.alg.__class__(*Kdim)            
        return self.__class__(self.alg.minus(self, other), alg_)

    def __rsub__(self, other):
        if isinstance(other, numbers.Number):
            other = self.alg(other)
        Kdim = self._getfield(self.alg,other.alg)
        alg_ = self.alg.__class__(*Kdim)
        return self.__class__(self.alg.minus(other, self), alg_)

    def __mul__(self, other):
        if isinstance(other, numbers.Number):
            other = self.alg(other)
        Kdim = self._getfield(self.alg,other.alg)
        alg_ = self.alg.__class__(*Kdim)            
        return self.__class__(self.alg.mul(self, other), alg_)

    def __rmul__(self, other):
        if isinstance(other, numbers.Number):
            other = self.alg(other)
        Kdim = self._getfield(self.alg,other.alg)
        alg_ = self.alg.__class__(*Kdim)        
        return self.__class__(self.alg.mul(other, self), alg_)

    def __neg__(self):
        return self.__class__(self.alg.neg(self), self.alg)

    def __eq__(self, other):
        return self.dp == other.dp and self.alg == other.alg

    def __call__(self, x):
        ''' Auswertung eines Polynoms p an der Stelle x mit p(x) '''
        return sum((pk*x**k for k, pk in self.dp.items()))

    def __pow__(self, n):
        """Potenzieren mit einer natürlichen Zahl"""
        if isinstance(n, int) and n >= 0:
            erg = self.__class__({0:1}, self.alg)
            for k in range(n):
                erg *= self
            return erg
        else:
            raise NotImplementedError()


class PolyGruppe:
    ''' 
        Polynome als additive Grupppe über Ring/Körper

        PolyGruppe(field_str) 

        field_str darf hier C, R oder Z sein
        für komplexe, reelle bzw. ganze Zahlen
    '''

    def __init__(self, field_str):
        self.field_str = field_str
        if field_str == 'C':
            self.field = numbers.Complex
        elif field_str == 'R':
            self.field = numbers.Real
        elif field_str == 'Z':
            self.field = numbers.Integral
        else:
            raise AssertionError(
                f" 'field_str' {field_str} muss 'C', 'R', oder 'Z' sein")

    def __repr__(self):
        return f'Gruppe ({self.field_str}[X],+)'

    def __call__(self, dp):
        if isinstance(dp, numbers.Number):
            dp = {0: dp}
        self.validate(dp)
        return Polynom(dp, self)

    def validate(self, dp):
        '''Test der Eingaben'''

        assert all(isinstance(x, int) and x >= 0 for x in dp.keys()), \
            "Die Exponenten (keys) müssen natürliche Zahlen (einschl. 0) sein."
        assert all((isinstance(v, self.field) for v in dp.values())), \
            f'Für {self} müssen alle Koeffizienten in {dp} "{self.field.__name__}" sein.'

    def __eq__(self, other):
        return self.field == other.field

    def add(self, p, q):
        ''' Polynomaddition'''
        erg = {n: p.dp.get(n, 0) + q.dp.get(n, 0)
               for n in set(p.dp) | set(q.dp)}
        return erg

    def minus(self, p, q):
        ''' Polynomsubtraktion'''
        erg = {n: p.dp.get(n, 0) - q.dp.get(n, 0)
               for n in set(p.dp) | set(q.dp)}
        return erg

    def neg(self, p):
        ''' additive Inverse '''
        erg = {k: -v for k, v in p.dp.items()}
        return erg


class PolyRing(PolyGruppe):
    ''' Polynome als Ring über Ring/Körper
        C[X], R[X] bzw Z[X]

        PolyRing(field_str) 

        field_str darf hier C, R oder Z sein
        für komplexe, reelle bzw. ganze Zahlen '''

    def __repr__(self):
        return f'{self.field_str}[X]'

    def mul(self, p, q):
        ''' Multiplikation im Polynomring'''
        erg = {}
        for i, pi in p.dp.items():
            for j, qj in q.dp.items():
                ipj = i+j
                if ipj in erg:
                    erg[ipj] += pi*qj
                else:
                    erg[ipj] = pi*qj
        return erg



if __name__ == '__main__':
    
    RX = PolyRing('R')
    p = RX({1:1,0:-1})
    print(p**2)
    #p = RX({2:1,0:-1}) # p(x) x^2-1

    #p.plot()
    
    #p.plot(xmin=-1)
    
    #fig = plt.figure(10)
    #p.plot(fig, xmax=2, style='r-')



Polynom X^{2}-2*X+1


In [244]:
import numpy as np
def LinUnabh_Poly(a):
    grad=max([p.degree for p in a])                               #Maximalen Grad der Polynome bestimmen
    matrix=np.zeros((grad+1,len(a)))                              #Matrix, die später die Koeffizienten enthält, erstellen
    for p in range(len(a)):                                       #Polynome durchgehen
        for i in range(grad+1):                                   #In jedem Polynom die Koeffizienten durchgehen
            if i in a[p].dp.keys():                               #Prüfen, ob der Koeffizient ungleich 0 ist (ist er gleich 0, so ist der Exponent nicht im Dictionary dp)
                matrix[i,p]=a[p].dp[i]                            #Wert in die Matrix übertragen
    return np.linalg.matrix_rank(matrix)==min(len(a),grad+1)      #Prüfen ob der Rang voll ist

In [245]:
LinUnabh_Poly([RX({2:1,0:-1}),RX({1:1,0:-1})])

True

In [246]:
LinUnabh_Poly([RX({2:1,0:-1}),RX({2:2,0:-2})])

False

In [247]:
B0=RX({4:1,3:-4,2:6,1:-4,0:1})
B1=RX({4:-4,3:12,2:-12,1:4})
B2=RX({4:6,3:-12,2:6})
B3=RX({4:-4,3:4})
B4=RX({4:1})

In [248]:
LinUnabh_Poly([B0,B1,B2,B3,B4])

True

# Aufgabe 2

In [249]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# Musterlösung für Aufgabe 33

import numpy as np


# Vorwärtssubstitution

def vorwaerts(L, b):
    n = L.shape[0]
    
    assert (n == L.shape[1] and n == b.shape[0] and b.shape[1] == 1),\
        "Dimensionen passen nicht."
    
    assert np.prod(L.diagonal()), "L ist nicht invertierbar."
    # alternativ:
    #assert (not(any(np.diag(L) == 0))), "L ist nicht invertierbar."
    # oder:
    # Die Überprüfung geht natürlich auch in der Schleife, in dem jedes
    # Diagonalelement überprüft wird.
    
    x = np.zeros_like(b, dtype=float)
    x[0] = b[0]/L[0, 0]
    for i in range(1, n):
        x[i] = (b[i] - L[i, :i] @ x[:i])/L[i, i]
        
    return x



# Rückwärtssubstitution

def rueckwaerts(R, b):
    n = R.shape[0]
    
    assert (n == R.shape[1] and n == b.shape[0] and b.shape[1] == 1),\
        "Dimensionen passen nicht."
    
    assert np.prod(R.diagonal()), "R ist nicht invertierbar."
    
    x = np.zeros_like(b, dtype=float)
    x[n-1] = b[n-1]/R[n-1, n-1]
    for i in range(n-2, -1, -1):
        x[i] = (b[i] - R[i, i+1:] @ x[i+1:])/R[i, i]
        
    return x




In [250]:
import scipy as sc
def lgsloeser(A,b):
    P,L,R=sc.linalg.lu(A)                    #LR-Zerlegung speichern
    z=P.T@b                                  #PLR=A, also LR=P^(-1)A=P.T@A, weil Permutationsmatrizen sind orthogonal. Also Ax=b genau dann wenn LRx=P.Tb:=z.
    y=vorwaerts(L,z.reshape(z.shape[0],1))   #Gleichung Ly=z nach y lösen
    return rueckwaerts(R,y)                  #Gleichung Rx=y nach x lösen und ausgeben

In [251]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# Aufgabenkontrolle11.py

import numpy as np


def Aufgabe42(testfunktion):
    '''Aufgabe40 testet automatisch, ob die übergebene Funktion "testfunktion"
    die Anforderungen aus Aufgabe 42 erfüllt.
    Dazu löst es mit "testfunktion" einige lineare Gleichungssysteme
    verschiedener Größe und überprüft das Residuum.
    '''
    print('')
    print('Ihre Funktion wird jetzt mit 4 Gleichungssystemen getestet.');
    print('Wenn die Norm des Residuums klein ist (<10^-8), ist der Test bestanden.')
    print('')
    
    AsUndBs = [
        (np.array([
            [ 0, 1, 2 ],
            [ 1, 2, 5 ],
            [ 2, 3, 4 ],
        ], dtype = float), np.random.rand(3, 1)),
        _generierteZufallsmatrixMitGuterKonditionUndVektor(5, float),
        _generierteZufallsmatrixMitGuterKonditionUndVektor(100, int),
        _generierteZufallsmatrixMitGuterKonditionUndVektor(100, float),
    ]
    texte = [
        "der Matrix\n{0}",
        "einer reellen {0.shape[0]} x {0.shape[1]} Matrix",
        "einer ganzzahligen {0.shape[0]} x {0.shape[1]} Matrix",
        "einer reellen {0.shape[0]} x {0.shape[1]} Matrix",
    ]
    
    tb = 0
    for (A, b), text in zip(AsUndBs , texte):
        text = text.format(A)
        print(f"Teste mit {text}.")
        x = testfunktion(A.copy(), b)
        residuum = A @ x - b
        resNorm = np.linalg.norm(residuum)
        print(f"Die Norm des Residuums ist {resNorm}.\n")
        if resNorm < 1e-8:
            tb += 1
    
    print(f'{tb} von {len(AsUndBs)} Tests erfolgreich')


def _generierteZufallsmatrixMitGuterKonditionUndVektor(N, typ):
    """
    Generiert eine Zufallsmatrix vorgegebener Größe und vorgegebenem Typs
    so, dass die Konditionszahl der Matrix gut ist.
    """
    while True:
        # Wir raten so lange Matrizen, bis die Kondition der geratenen Matrix
        # gut ist.
        A = 200 * ((np.random.rand(N, N) - 1/2) * 2)
        A = A.astype(typ)
        if typ == complex:
            # Rate zusätzlichen Imaginärteil (brauchen wir hier nicht)
            A += 1j * 200 * ((np.random.rand(N, N) - 1/2) * 2)
        if np.linalg.cond(A) <= N:
            break
    b = np.random.rand(N, 1)
    return A, b
    
    
def TestLR_Bruch(testfunktion):
    m = 10
    print(f'Ihre Funktion wird jetzt mit 3 ({m}x{m})-Matrizen getestet...');
    A = _zufallsmatINT(m)
    B = _zufallsmatBRUCH(m)
    C1 = _zufallsmatINT(m)
    C = _zufallsmatBRUCH(m)
    for i in range(m):
        for j in range(m):
            zufallsindex = np.random.randint(0,2)
            if zufallsindex == 0:
                C[i, j] = C1[i, j]
    texte = ['Matrix mit Integer-Einträgen',\
             'Matrix mit Bruch-Einträgen',\
             'Matrix mit gemischten Einträgen']
    tb = 0
    for M, text in zip([A, B, C], texte):
        print('Teste ' + text + ':')
        p, MLR = testfunktion(M)
        R = np.triu(MLR)
        L = np.eye(MLR.shape[0], dtype=int) + np.tril(MLR, -1)
        erg = np.equal(M[p,:], L@R).all()
        print(f'Das Ergebnis ist {"richtig" if erg else "falsch"}.');
        if erg:
            tb += 1;
    
    print(f'{tb} von 3 Tests erfolgreich.');

def _zufallsmatINT(m, Null = True):
    mat = np.random.randint(-20,20, m**2)
    if Null == False:
        mat[mat == 0] += 1
    return mat.reshape(m,m)

def _zufallsmatBRUCH(m):
    ZufINT1 = _zufallsmatINT(m)
    ZufINT2 = _zufallsmatINT(m, Null = False)
    MatBRUCH = np.zeros((m,m), dtype=Bruch)
    for i in range(m):
        for j in range(m):
            MatBRUCH[i,j] = Bruch(int(ZufINT1[i,j]), int(ZufINT2[i,j]))
    return MatBRUCH



In [252]:
Aufgabe42(lgsloeser)


Ihre Funktion wird jetzt mit 4 Gleichungssystemen getestet.
Wenn die Norm des Residuums klein ist (<10^-8), ist der Test bestanden.

Teste mit der Matrix
[[0. 1. 2.]
 [1. 2. 5.]
 [2. 3. 4.]].
Die Norm des Residuums ist 1.1102230246251565e-16.

Teste mit einer reellen 5 x 5 Matrix.
Die Norm des Residuums ist 1.5700924586837752e-16.

Teste mit einer ganzzahligen 100 x 100 Matrix.
Die Norm des Residuums ist 5.71057562494717e-14.

Teste mit einer reellen 100 x 100 Matrix.
Die Norm des Residuums ist 3.9538092681129646e-14.

4 von 4 Tests erfolgreich


# Aufgabe 3

In [253]:
#Aufgabe 43a:

#Die Methoden __radd__,__rmul__,... sind dazugekommen. 
#Sie erlauben rechnen mit Skalaren auch in verkehrter Reihenfolge.
#Vorher durfe bei a+b nur b auch ein ein Integer sein.
#Das r in __radd__,... steht wahrscheinlich für reverse.
#Die Methode __eq__ ist dazugekommen.
#Sie testet Gleichheit von Brüchen.
#Die Methode __pow__ ist dazugekommen.
#Sie erlaubt das Potenzieren von Brüchen.

In [254]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Stand: Donnerstag, 19.12.

"""

# In diesem Modul finden Sie die verbesserte und erweiterte Musterlösung zur
# Bruchklasse aus Aufgabe 15.



# Sie können die Datei als Skript ausführen, oder als Modul z.B. mit
# import bruch as beliebigerName
# oder nur die Klasse Bruch mit
# from bruch import Bruch
# laden.


class Bruch():
    '''Bruchklasse'''

    from math import gcd

    def __init__(self, zaehler=0, nenner=1):
        '''Initialisieren, Zähler und Nenner können als Integer oder Objekte
        der Klasse Bruch übergeben werden'''
        if type(nenner) is int:
            if type(zaehler) is int:
                self.zaehler = zaehler
                self.nenner = nenner
            elif isinstance(zaehler, Bruch):
                self.zaehler = zaehler.zaehler
                self.nenner = zaehler.nenner * nenner
            else:
                raise NotImplementedError
        elif isinstance(nenner, Bruch):
            if type(zaehler) is int:
                self.zaehler = zaehler * nenner.nenner
                self.nenner = nenner.zaehler
            elif isinstance(zaehler, Bruch):
                self.zaehler = zaehler.zaehler * nenner.nenner
                self.nenner = zaehler.nenner * nenner.zaehler
            else:
                raise NotImplementedError
        elif nenner == 0:
            raise ZeroDivisionError
        else:
            raise NotImplementedError

        if self.nenner < 0:
            self.zaehler, self.nenner = self.__kuerzen__(
                -self.zaehler, -self.nenner)
        else:
            self.zaehler, self.nenner = self.__kuerzen__(
                self.zaehler, self.nenner)

    def __kuerzen__(self, zaehler, nenner):
        '''Zähler und Nenner kürzen'''
        # Die Unterstriche in der Benennung sorgen dafür, dass die Funktion
        # versteckt ist. D.h. definiert man außerhalb einen Bruch a, zeigt die
        # Eingabe von a. und (TAB) nur nicht-versteckte Methoden, wie z.B.
        # tofloat an, aber nicht die Methode kuerzen. Das ist hier sinnvoll,
        # weil intern immer ein gekürzter Bruch gespeichert werden soll und wir
        # außerhalb die Methode kuerzen niemals ausführen werden.
        ggT = self.gcd(zaehler, nenner)
        # ganzzahlige Division
        return zaehler//ggT, nenner//ggT

    def __add__(self, other):
        '''Methode zum Addieren (a+b), wobei a ein Bruch und b ein Integer 
        oder Bruch ist'''
        if isinstance(other, int):
            other = Bruch(other)
        zaehler = self.zaehler*other.nenner + other.zaehler*self.nenner
        nenner = self.nenner*other.nenner
        return Bruch(zaehler, nenner)

    def __sub__(self, other):
        '''Methode zum Subtrahieren (a-b), wobei a ein Bruch und b ein Integer 
        oder Bruch ist'''
        if isinstance(other, int):
            other = Bruch(other)
        return self + Bruch(-other.zaehler, other.nenner)

    def __truediv__(self, other):
        '''Methode zum Dividieren (a/b), wobei a ein Bruch und b ein Integer 
        oder Bruch ist'''
        if isinstance(other, int):
            other = Bruch(other)
        zaehler = self.zaehler * other.nenner
        nenner = self.nenner * other.zaehler
        return Bruch(zaehler, nenner)

    def __mul__(self, other):
        '''Methode zum Multiplizieren (a*b), wobei a ein Bruch und b ein Integer 
        oder Bruch ist'''
        if isinstance(other, int):
            other = Bruch(other)
        zaehler = self.zaehler * other.zaehler
        nenner = self.nenner * other.nenner
        return Bruch(zaehler, nenner)

    # Umgekehrte Multiplikation, so dass auch Skalar*Bruch funktioniert:
    __rmul__ = __mul__

    # Umgekehrte Addition, so dass auch Skalar+Bruch funktioniert:
    __radd__ = __add__

    def __rsub__(self, other):
        '''Methode für umgekehrte Subtraktion (b-a), wobei a ein Bruch und b
        ein Integer oder Bruch ist'''
        if isinstance(other, int):
            other = Bruch(other)
        return other - self

    def __rtruediv__(self, other):
        '''Methode für umgekehrte Divison b/a, wobei a ein Bruch und b
        ein Integer oder Bruch ist'''
        if isinstance(other, int):
            other = Bruch(other)
        return other / self

    def __eq__(self, other):
        '''Methode zum Prüfen von Gleichheit (a==b), wobei a und b Brüche, 
        Floats oder Integer sein können'''
        if isinstance(other, (int, float)):
            return self.zaehler == self.nenner * other
        else:
            return self.zaehler*other.nenner == other.zaehler*self.nenner

    def __pow__(self, skalar):
        '''Methode zum Potenzieren (a**n) für Bruch a und ganze Zahl n'''
        if isinstance(skalar, int) and skalar >= 0:
            return Bruch(self.zaehler**skalar, self.nenner**skalar)
        elif isinstance(skalar, int) and skalar < 0:
            return Bruch(self.nenner**(-skalar), self.zaehler**(-skalar))
        else:
            raise NotImplementedError

    def tofloat(self):
        '''Methode zum Umwandeln des Bruchs in einen Float '''
        return self.zaehler/self.nenner

    ''' Die folgenden beiden Methoden dienen dazu, dass Sie den Bruch sehen 
    wenn Sie ein Bruch-Objekt in die Konsole tippen oder print() übergeben.
    '''

    def __str__(self):
        return '{:d}/{:d}'.format(self.zaehler, self.nenner)

    def __repr__(self):
        return '{:d}/{:d}'.format(self.zaehler, self.nenner)



    #Aufgabe 43b:

    

    def __lt__(self,other):
        return self.zaehler*Bruch(other).nenner<self.nenner*Bruch(other).zaehler    #Auf Hauptnenner erweitern, Methodennamen standen auf Website, Bruch um other, weil es ein int sein könnte.

    def __le__(self,other):
        return self.zaehler*Bruch(other).nenner<=self.nenner*Bruch(other).zaehler

    def __gt__(self,other):
        return self.zaehler*Bruch(other).nenner>self.nenner*Bruch(other).zaehler     

    def __ge__(self,other):
        return self.zaehler*Bruch(other).nenner>=self.nenner*Bruch(other).zaehler

    def __abs__(self):
        if self.zaehler>=0:
            return self
        else:
            return Bruch(-self.zaehler,self.nenner)                    #Wenn Zaehler negativ ist, gilt abs(Bruch)=-Bruch, sonst abs(Bruch)=Bruch.




if __name__ == '__main__':

    a = Bruch(5, -2)
    b = Bruch(8, 4)
    c = Bruch(0, 9)
    print('\n---Ein paar Tests---\n')
    print('gekürztes b:', b)
    print('Multiplikation a*c:', a*c)
    print('Addition a+b:', a+b)
    print('Addition a+3:', a+3)
    print('Division:', a/b)
    print('a gleich b?', a == b)
    print('b gleich 2?', b == 2)
    print('a hoch -3:', a**(-3))




---Ein paar Tests---

gekürztes b: 2/1
Multiplikation a*c: 0/1
Addition a+b: -1/2
Addition a+3: 1/2
Division: -5/4
a gleich b? False
b gleich 2? True
a hoch -3: -8/125


In [255]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-




#Aufgabe 43c:




# Musterlösung für Aufgabe 35

import numpy as np

def intify(A):                                                         #np.int32 ist komisch und gibt oft Fehlermeldungen, kommt aber manchmal bei Rechnungen in die Matrix. Das wird hier aufgehoben.
    n = A.shape[0]
    for i in range(n):                                                 #Matrixeinträge durchgehen, falls sie np.int32 sind, werden sie zu int gemacht.
        for j in range(n):
            if isinstance(A[i,j],np.int32):
                A[i,j]=int(A[i,j])
                
def LR_kompakt(A):
    A = A.astype(Bruch).copy()                                         #Genau wie in der Musterlösung, nur dass jetzt mit Brüchen statt floats gerechnet wird.
    n = A.shape[0]
    p = np.arange(n)
    
    for kk in range(n-1):
        intify(A)                                                      #np.int32 loswerden
        ll = kk + np.argmax(abs(A[kk:, kk]))
        A[[kk, ll]] = A[[ll, kk]]
        p[[kk, ll]] = p[[ll, kk]]
        for i in range(kk+1,n):                                        #Einzige wirkliche Änderung: Division mit __truediv__ eintragsweise, wobei der Zähler erst zum Bruch gemacht werden muss, falls er int ist.
            A[i, kk] = Bruch(A[i, kk]).__truediv__(A[kk, kk])
        A[kk + 1:, kk+1:] -= A[kk +1:, [kk]] @ A[[kk], kk + 1:]        #Rest genau wie vorher
        
    return p, A

In [256]:
TestLR_Bruch(LR_kompakt)

Ihre Funktion wird jetzt mit 3 (10x10)-Matrizen getestet...
Teste Matrix mit Integer-Einträgen:
Das Ergebnis ist richtig.
Teste Matrix mit Bruch-Einträgen:
Das Ergebnis ist richtig.
Teste Matrix mit gemischten Einträgen:
Das Ergebnis ist richtig.
3 von 3 Tests erfolgreich.


# Aufgabe 4

In [257]:
#Einfach der übertragene Pseudocode.
def GramSchmidt(A):
    n = A.shape[0]
    R = np.zeros((n,n))
    q = np.zeros((n,n))
    for k in range(n):
        q[:,k] = A[:,k]
        if not k == 0:
            summe=np.zeros(n)
            for j in range(k):
                R[j,k] = np.dot(q[:,j],q[:,k])
                summe += R[j,k]*q[:,j]
            q[:,k] -= summe
        R[k,k] = np.linalg.norm(q[:,k])
        q[:,k] /= R[k,k]
    return q,R

In [258]:
#Einfach der übertragene Pseudocode.
def GramSchmidt_mod(A):
    n = A.shape[0]
    R = np.zeros((n,n))
    q = np.zeros((n,n))
    for k in range(n):
        q[:,k] = A[:,k]
        for i in range(k):
            R[i,k] = np.dot(q[:,i],q[:,k])
            q[:,k] -= R[i,k]*q[:,i]
        R[k,k] = np.linalg.norm(q[:,k])
        q[:,k] /= R[k,k]
    return q,R

In [259]:
N=10                                                   #Größe der Testmatrix
A=200 * ((np.random.rand(N, N) - 1/2) * 2)             #Zufällige Float-Testmatrix (geklaut aus Aufgabenkontrolle11)
q,R=GramSchmidt(A)                                     #Auswertung der Methoden
q2,R2=GramSchmidt_mod(A)
print(np.linalg.det(q))                                #Hier sieht man den Unterschied zwischen den Methoden - die Determinante kann leicht anders sein. Immer 1 oder -1, weil q Orthogonal ist.
print(np.linalg.det(q2))
print(np.allclose(q.T@q,np.eye(N,N)))                  #Testet, ob q und q2 wirklich Orthogonalmatrizen sind, die Spalten also eine Orthonormalbasis bilden.
print(np.allclose(q2.T@q2,np.eye(N,N)))
print(np.allclose(q@R,A))                              #Testet, ob die Matrix R tatsächlich wie gewollt die Gleichung qR=A erfüllt.
print(np.allclose(q2@R2,A))

1.0
1.0000000000000002
True
True
True
True
